##**Note**: Change Runtime type to "GPU" before connecting to runtime or you will face issues

Credits for our MCNN baseline model: https://www.kaggle.com/code/wptouxx/shtech-mcnn

In [1]:
import warnings
warnings.filterwarnings("ignore")
!pip install opendatasets --upgrade --quiet
import opendatasets as od
od.version()

'0.1.22'

In [2]:
!git clone https://github.com/nksrivastavaa/DL_project_files

Cloning into 'DL_project_files'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), 15.78 MiB | 8.97 MiB/s, done.


In [3]:
!unzip /content/DL_project_files/ShanghaiTech.zip

Archive:  /content/DL_project_files/ShanghaiTech.zip
   creating: ShanghaiTech/part_A/
   creating: ShanghaiTech/part_A/test_data/
   creating: ShanghaiTech/part_A/test_data/ground-truth/
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_1.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_10.mat  
  inflating: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_2.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_3.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_4.mat  
  inflating: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_5.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_6.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_7.mat  
  inflating: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_8.mat  
 extracting: ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_9.mat  
   creating: ShanghaiTech/part_A/test_data/images/
  inflating: ShanghaiTech/part_A/test_data/images/

In [4]:
!pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 693.7 kB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np

In [7]:
import scipy
import scipy.io
import cv2

from matplotlib import pyplot as plt

from scipy.ndimage import gaussian_filter

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from PIL import Image

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning import seed_everything

import os

In [8]:
from sklearn.model_selection import train_test_split

train = [p.path for p in os.scandir('/content/ShanghaiTech/part_A/train_data/images/')]
valid_full = [p.path for p in os.scandir('/content/ShanghaiTech/part_A/test_data/images/')]

## use a small subset for validation
_, valid = train_test_split(valid_full, test_size=2, random_state=42)

len(train), len(valid)

(30, 2)

In [9]:
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [36]:
im_size = 300
aug_train = A.Compose([
    A.RandomCrop(im_size, im_size),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(),
    A.Normalize((0.5), (0.5)),
], keypoint_params=A.KeypointParams(format='xy', angle_in_degrees=False))

aug_val = A.Compose([
    A.Resize(768, 1024),
    A.Normalize((0.5), (0.5)),
], keypoint_params=A.KeypointParams(format='xy', angle_in_degrees=False))

class MyDataset(Dataset):
    def __init__(self, files, aug):
        self.files = files
        self.aug = aug
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        fn = self.files[idx]
        
        im = cv2.imread(fn, cv2.IMREAD_COLOR)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        
        m = scipy.io.loadmat(fn.replace('images', 'ground-truth').replace('IMG', 'GT_IMG').replace('.jpg', '.mat'))
        ps = m['image_info'][0][0][0][0][0]
        
        rst = self.aug(image=im, keypoints=ps)
        im = rst['image']
        ps = rst['keypoints']
        
        dm = np.zeros((im.shape[0], im.shape[1]), dtype=np.float32)
        for x, y in ps:
            x = int(x)
            y = int(y)
            dm[y, x] = 1

        sigma = 4
        dm = gaussian_filter(dm, sigma=sigma, truncate=4*sigma)
        
        dm = cv2.resize(dm, (im.shape[1] // 4, im.shape[0] // 4), interpolation=cv2.INTER_LINEAR)
        dm *= 16
        
        im = torch.from_numpy(im)
        dm = torch.from_numpy(dm)
        
        return im, dm

In [37]:
class Conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, relu=True, same_padding=False, bn=False):
        super(Conv2d, self).__init__()
        padding = int((kernel_size - 1) / 2) if same_padding else 0
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001, momentum=0, affine=True) if bn else None
        self.relu = nn.ReLU(inplace=True) if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

In [38]:
class MCNN(LightningModule):
    '''
    Multi-column CNN 
        -Implementation of Single Image Crowd Counting via Multi-column CNN (Zhang et al.)
    '''
    
    def __init__(self, lr, batch_size, max_steps, bn=False):
        super(MCNN, self).__init__()
        
        self.lr = lr
        self.save_hyperparameters()
        
        self.use = 0
        
        self.branch1 = nn.Sequential(Conv2d( 1, 16, 9, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(16, 32, 7, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(32, 16, 7, same_padding=True, bn=bn),
                                     Conv2d(16,  8, 7, same_padding=True, bn=bn))
        
        self.branch2 = nn.Sequential(Conv2d( 1, 20, 7, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(20, 40, 5, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(40, 20, 5, same_padding=True, bn=bn),
                                     Conv2d(20, 10, 5, same_padding=True, bn=bn))
        
        self.branch3 = nn.Sequential(Conv2d( 1, 24, 5, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(24, 48, 3, same_padding=True, bn=bn),
                                     nn.MaxPool2d(2),
                                     Conv2d(48, 24, 3, same_padding=True, bn=bn),
                                     Conv2d(24, 12, 3, same_padding=True, bn=bn))
        
        self.fuse = nn.Sequential(Conv2d( 30, 1, 1, same_padding=True, bn=bn))
        
        self.out1 = nn.Sequential(Conv2d( 8, 1, 1, same_padding=True, bn=bn))
        self.out2 = nn.Sequential(Conv2d( 10, 1, 1, same_padding=True, bn=bn))
        self.out3 = nn.Sequential(Conv2d( 12, 1, 1, same_padding=True, bn=bn))
        
        self.crit = nn.MSELoss()
        
    def forward(self, im_data):
        im_data = im_data.unsqueeze(1)
        x1 = self.branch1(im_data)
        x2 = self.branch2(im_data)
        x3 = self.branch3(im_data)
        
        
        if self.use == 0:
            x = torch.cat((x1,x2,x3),1)
            x = self.fuse(x)
        elif self.use == 1:
            x = self.out1(x1)
        elif self.use == 2:
            x = self.out2(x2)
        elif self.use == 3:
            x = self.out3(x3)
        
        return x.squeeze(1)
    
    
    def training_step(self, batch, batch_idx):
        self.train()
        x, y = batch
        
        pred = self(x)
        loss = self.crit(pred, y)
        
        pred_sum = torch.round(pred.sum(dim=(1,2))).int()
        gt_sum = torch.round(y.sum(dim=(1,2))).int()
        acc = (pred_sum == gt_sum).float().mean()
        
        mae = torch.abs(pred_sum - gt_sum).float().mean()
        
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        self.log('train_mae', mae)
        
        return loss
        
    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            self.eval()
            x, y = batch
            
            pred = self(x)
            loss = self.crit(pred, y)
        
            pred_sum = torch.round(pred.sum(dim=(1,2))).int()
            gt_sum = torch.round(y.sum(dim=(1,2))).int()
            acc = (pred_sum == gt_sum).float().mean()

            mae = torch.abs(pred_sum - gt_sum).float().mean()
            
            self.log('val_loss', loss)
            self.log('val_acc', acc)
            self.log('val_mae', mae)
            
            
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=1e-4)
        
        scheduler = {
            'scheduler': torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=self.lr, total_steps=self.hparams.max_steps, pct_start=0.1, cycle_momentum=False),
            'interval': 'step',
            'frequency': 1
        }
        
        return [optimizer], [scheduler]

In [39]:
def weights_normal_init(model, dev=0.01):
    if isinstance(model, list):
        for m in model:
            weights_normal_init(m, dev)
    else:
        for n, m in model.named_modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(0.0, dev)
                if m.bias is not None:
                    m.bias.data.fill_(0.0)
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0.0, dev)

In [47]:
batch_size = 4
epochs = 100
max_steps = epochs * len(train) // batch_size

In [49]:
train_loader = DataLoader(MyDataset(train, aug_train), batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(MyDataset(valid, aug_val), batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)

#MCNN MODEL TRAINING

In [50]:
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
seed_everything(42)

checkpoint_cb = ModelCheckpoint(
    save_top_k=1,
    save_last=True,
    verbose=True,
    monitor='val_mae',
    mode='min'
)

trainer = Trainer(max_steps=max_steps, precision=16, benchmark=True, callbacks=[checkpoint_cb, LearningRateMonitor()])

lr = 3e-4

model = MCNN(lr, batch_size, max_steps)
weights_normal_init(model, dev=0.01)

model.use = 0

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type       | Params
---------------------------------------
0 | branch1 | Sequential | 57.8 K
1 | branch2 | Sequential | 46.1 K
2 | branch3 | Sequential | 24.0 K
3 | fuse    | Sequential | 31    
4 | out1    | Sequential | 9     
5 | out2    | Sequential | 11    
6 | out3    | Sequential | 13    
7 | crit    | MSELoss    | 0     
---------------------------------------
127 K     Tra

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 7: 'val_mae' reached 324.00000 (best 324.00000), saving model to '/content/lightning_logs/version_5/checkpoints/epoch=0-step=7.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 104, global step 735: 'val_mae' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 105, global step 742: 'val_mae' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 106, global step 749: 'val_mae' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 107, global step 750: 'val_mae' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=750` reached.


# `Loading model from last checkpoint`

Credits: https://www.kaggle.com/code/wptouxx/shtech-mcnn-eval/notebook

In [51]:
from tqdm.notebook import tqdm

In [52]:
model = MCNN.load_from_checkpoint('/content/lightning_logs/version_5/checkpoints/epoch=4-step=35.ckpt')
model = model.cuda()
model.eval()
rst = []
gt = []
for x, y in tqdm(val_loader):
    with torch.no_grad():
        x = x.cuda(non_blocking=True)
        pred = torch.round(model(x).sum(dim=(1,2))).int().cpu().numpy()
        
    rst.append(pred)
    gt.append(y.sum(dim=(1,2)).int().numpy())
    
rst = np.concatenate(rst, axis=0)
gt = np.concatenate(gt, axis=0)

  0%|          | 0/1 [00:00<?, ?it/s]

#MAE

In [53]:
print("MAE:", np.abs((rst - gt)).astype(np.float32).mean())

MAE: 54.0


#MSE

In [54]:
print("MSE:", ((rst - gt) ** 2).mean())

MSE: 3037.0


Convert lightning_logs to zip file so that it can be downloaded

In [55]:
!zip -r /content/lightning_logs.zip /content/lightning_logs

  adding: content/lightning_logs/ (stored 0%)
  adding: content/lightning_logs/version_3/ (stored 0%)
  adding: content/lightning_logs/version_3/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_3/events.out.tfevents.1679226034.28871191401c.829.3 (deflated 31%)
  adding: content/lightning_logs/version_5/ (stored 0%)
  adding: content/lightning_logs/version_5/checkpoints/ (stored 0%)
  adding: content/lightning_logs/version_5/checkpoints/epoch=4-step=35.ckpt (deflated 8%)
  adding: content/lightning_logs/version_5/checkpoints/last.ckpt (deflated 8%)
  adding: content/lightning_logs/version_5/events.out.tfevents.1679226254.28871191401c.829.5 (deflated 68%)
  adding: content/lightning_logs/version_5/hparams.yaml (stored 0%)
  adding: content/lightning_logs/version_4/ (stored 0%)
  adding: content/lightning_logs/version_4/checkpoints/ (stored 0%)
  adding: content/lightning_logs/version_4/checkpoints/epoch=0-step=15.ckpt (deflated 7%)
  adding: content/lightning_logs/versio

Download lightning_logs.zip

In [56]:
from google.colab import files
files.download("/content/lightning_logs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [57]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.1
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
bokeh                         2.4.3
branca                        0.6.0
bs4                           0.0.1
CacheControl                  0.12.11
cach